In [9]:
from tqdm.notebook import tqdm
import logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(format=' %(asctime)s - %(levelname)s - %(message)s')

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [10]:
from contextlib import ExitStack
from waiting import wait

from tapearchive import app
from tapearchive.utils import get_config

app_config = get_config("./config.yaml")
exit_stack = ExitStack()
connection_pool = app.create_db_connection(app_config)
dispatcher = app.create_dispatcher(connection_pool, app_config, exit_stack)

 2022-06-19 18:12:59,580 - DEBUG - Creating 7 workers
 2022-06-19 18:12:59,664 - DEBUG - Worker <Worker(Thread-19, started daemon 139724308866624)> starts
 2022-06-19 18:12:59,665 - DEBUG - Worker <Worker(Thread-20, started daemon 139724300473920)> starts
 2022-06-19 18:12:59,665 - DEBUG - Worker <Worker(Thread-21, started daemon 139724292081216)> starts
 2022-06-19 18:12:59,665 - DEBUG - Worker <Worker(Thread-22, started daemon 139723540526656)> starts
 2022-06-19 18:12:59,665 - DEBUG - Worker <Worker(Thread-23, started daemon 139723532133952)> starts
 2022-06-19 18:12:59,666 - DEBUG - Worker <Worker(Thread-24, started daemon 139723523741248)> starts
 2022-06-19 18:12:59,667 - DEBUG - Worker <Worker(Thread-25, started daemon 139723515348544)> starts
 2022-06-19 18:12:59,667 - DEBUG - Dispatch loop starting.
 2022-06-19 18:12:59,670 - DEBUG - <class 'tapearchive.workflow.tasks.audio_convert.ConvertAudio'> had been registered to <tapearchive.workflow.tasks.audio_convert.AudioConverterHa

In [11]:
from tapearchive.workflow.tasks.audio_analisis import FindTuneKey
from tapearchive.models.raw_data import FileDao

import pathlib 
import glob

source_path = pathlib.Path("/home/caiwan/hdd/**/*.*")\

file_dao = FileDao(connection_pool)

for file in tqdm(glob.glob(str(source_path), recursive=True)):
    input_file = pathlib.Path(file)
    if input_file.is_file():
        file_id = file_dao.pull_from_disk(input_file)

        print(input_file)
        
        dispatcher.post_task(FindTuneKey(
            source_file_id=file_id,
            source_format=input_file.suffix
        ))


        break

  0%|          | 0/13044 [00:00<?, ?it/s]

 2022-06-19 18:12:59,968 - DEBUG - Worker <Worker(Thread-22, started daemon 139723540526656)> starts
 2022-06-19 18:12:59,970 - DEBUG - dispatch loop_tick
 2022-06-19 18:13:00,460 - DEBUG - Task posted: FindTuneKey(source_file_id=UUID('47b3210a-df81-4c0a-8823-294ceb7868b6'), source_format='.flac')


/home/caiwan/hdd/2 Raff - Don't Stop The Music [CUjoPOQNeRA].flac


In [12]:
logger.debug("------------")

dispatcher.terminate()
wait(lambda: dispatcher.is_exit)

exit_stack.close()

 2022-06-19 18:13:00,522 - DEBUG - ------------
 2022-06-19 18:13:00,527 - DEBUG - Task posted: <tq.task_dispacher.TerminateDispatcherLoop object at 0x7f14210b64f0>


KeyboardInterrupt: 